# Import Libraries

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf

import torchaudio
from torchaudio import transforms

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
root_dir = '../input/birdclef-2022'
train_df = pd.read_csv(root_dir + '/train_metadata.csv')
train_df.head()

# Quick EDA and Data Pre-processing

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

Looks like no null values are there.

In [ ]:
# For this baseline notebook, we would consider following columns 
columns_of_interest = ['primary_label', 'rating', 'filename']
train_df = train_df[columns_of_interest]
train_df.head()

In [ ]:
f, ax = plt.subplots(figsize=(20,8))
sns.countplot(x='rating', data=train_df, ax=ax)

we will consider only those audio files whose rating >= 3.0

In [ ]:
train_df = train_df[train_df.rating >= 3.0]

In [ ]:
train_df = train_df.reset_index(drop=True)
train_df.head()

In [ ]:
labels = train_df.primary_label.unique().tolist()

In [ ]:
label2id = {labels[i]: i for i in range(len(labels))}

In [ ]:
labels[0], label2id['afrsil1']

In [ ]:
train_audio_dir = '../input/birdclef-2022/train_audio'

In [ ]:
for i in tqdm(range(len(train_df))):
    filename = train_df.loc[i,'filename']
    sig, sr = torchaudio.load(os.path.join(train_audio_dir, filename))
    train_df.loc[i, 'num_channels'] = sig.shape[0]  # number of audio channels (mono/stereo)
    train_df.loc[i, 'signal_len'] = sig.shape[1]  # signal length
    train_df.loc[i, 'sampling_rate'] = sr

In [ ]:
train_df['num_channels'] = train_df['num_channels'].astype('int64')
train_df['signal_len'] = train_df['signal_len'].astype('int64')
train_df['sampling_rate'] = train_df['sampling_rate'].astype('int64')
train_df.head()

In [ ]:
train_df.sampling_rate.unique()

we have unqiue sampling rate i.e., 32 KHz

For a 1 second video, the array size will be 32000

In [ ]:
sns.countplot(train_df.num_channels)

There are more number of two channel audios. So we will convert mono (1 channel ) audio to stereo (2 channel) audio by replication.

In [ ]:
def MonoToStereo(aud, num_channel=2):
    sig, sr = aud
    if sig.shape[0] == num_channel:
        return aud
    else:
        stereo_sig = torch.cat([sig, sig])
    
    return (stereo_sig, sr)

In [ ]:
train_df.signal_len.min(), train_df.signal_len.max()

In [ ]:
f, ax = plt.subplots(figsize=(15,8))
sns.distplot(train_df.signal_len, ax=ax)

we need to have signal length to be of same size. So we will either pad the signal or truncate the signal.

In [ ]:
# Let's consider the length of all videos to be 10 seconds (A hyperparam - to be tuned)
max_len_ms = 10000 # 10k milliseconds ~ 10 seconds

def pad_signal(aud, max_len_ms):
    sig, sr = aud
    num_channels, sig_len = sig.shape
    max_len = sr // 1000 * max_len_ms
    
    if sig_len > max_len:
        sig = sig[:, :max_len]
    elif sig_len < max_len:
        # padding both sides of the signal
        pad_begin_len = random.randint(0, max_len - sig_len)
        pad_end_len = max_len - sig_len - pad_begin_len
        
        # pad with zeros
        pad_begin = torch.zeros((num_channels, pad_begin_len))
        pad_end = torch.zeros((num_channels, pad_end_len))
        
        sig = torch.cat((pad_begin, sig, pad_end), 1)
    
    return (sig, sr)

In [ ]:
# Data Augmentation on raw audio: Time shift
def time_shift(aud, shift_limit):
    sig, sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

In [ ]:
# Convert augmented audio to Mel Spectrogram
def mel_spec(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig, sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    # shape of spec: (channels, n_mels, time)
    
    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return spec

In [ ]:
# Data augmentation on mel spectrogram: Time and Frequency Masking
def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec
    
    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
        aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)
        
    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
        aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
        
    return aug_spec

Preprocessing on one signal sample

In [ ]:
idx = random.randint(0, len(train_df))
filename = train_df.loc[idx,'filename']
sig, sr = torchaudio.load(os.path.join(train_audio_dir, filename))
audio_sample = (sig, sr)

# mono to stereo (if mono)
audio = MonoToStereo(audio_sample)

# Pad or Truncate
audio = pad_signal(audio, max_len_ms)

# Augment on raw audio
audio = time_shift(audio, shift_limit=0.4)

# Convert to Mel Spectrogram
spec = mel_spec(audio)

# Augment on mel spec
aug_spec = spectro_augment(spec)
aug_spec.shape

In [ ]:
aug_spec_np = aug_spec.permute(1,2,0).numpy()
f, ax = plt.subplots(figsize=(15,8))
plt.imshow(aug_spec_np[:, :, 0])
plt.show()
f, ax = plt.subplots(figsize=(15,8))
plt.imshow(aug_spec_np[:, :, 1])
plt.show()

# Building Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, audio_dir, df, max_len_ms=10000, shift_limit=0.4):
        self.train_audio_dir = audio_dir
        self.train_df = df
        self.max_len_ms = max_len_ms
        self.shift_limit = shift_limit
        
    def __len__(self):
        return len(self.train_df)

    def __getitem__(self, index):
        filename = self.train_df.loc[index, 'filename']
        label = self.train_df.loc[index, 'primary_label']
        
        sig, sr = torchaudio.load(os.path.join(self.train_audio_dir, filename))
        audio = (sig, sr)
        
        ##################        
        # process signal #
        ##################
        
        # mono to stereo (if mono)
        audio = MonoToStereo(audio)

        # Pad or Truncate
        audio = pad_signal(audio, self.max_len_ms)

        # Augment raw audio
        audio = time_shift(audio, shift_limit=self.shift_limit)

        # Convert to Mel Spectrogram
        spec = mel_spec(audio)

        # Augment mel spec
        aug_spec = spectro_augment(spec)
        
        return aug_spec, torch.tensor(label2id[label])

In [ ]:
train_audio_dir = '../input/birdclef-2022/train_audio'
max_len_ms = 10000
shift_limit = 0.4
dataset = CustomDataset(train_audio_dir, train_df, max_len_ms, shift_limit)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
inputs, targets = next(iter(train_loader))
inputs.shape, targets.shape

Cool!!

Let's build model architecture.

# Model Building

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(2,8,3,1,1,bias=False)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,16,3,1,1,bias=False)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16,32,3,1,1,bias=False)
        self.bn3 = nn.BatchNorm2d(32)
        
        self.fc1 = nn.Linear(19968, 1000)
        self.fc2 = nn.Linear(1000, num_classes)
        
        self.pool = nn.MaxPool2d(2,2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = CNN(num_classes=len(labels))
output = model(inputs)
output.shape

In [ ]:
LERANING_RATE = 1e-4

# Defining loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LERANING_RATE)

# Model Training

In [ ]:
def train_epoch(model, data_loader, device, criterion, optimizer):
    model.train()

    losses = []
    correct = 0
    total = 0

    for batch_idx, (x, y) in enumerate(tqdm(data_loader)):
        x = x.to(device) 
        y = y.to(device) 

        output = model(x)

        loss = criterion(output, y)

        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, preds = torch.max(output, dim=1)
        correct += (preds.cpu() == y.cpu()).sum().item()
        total += preds.size(0)

    acc = (correct * 1.0) / total

    return acc, np.mean(losses)

In [ ]:
def val_epoch(model, data_loader, device, criterion):
    model.eval()

    losses = []
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(tqdm(data_loader)):
            x = x.to(device) 
            y = y.to(device) 

            output = model(x)

            loss = criterion(output, y)

            losses.append(loss.item())

            _, preds = torch.max(output, dim=1)
            correct += (preds.cpu() == y.cpu()).sum().item()
            total += preds.size(0)

    acc = (correct * 1.0) / total

    return acc, np.mean(losses)

In [ ]:
def train(model, epochs, device, train_loader, val_loader, criterion, optimizer):
    history = defaultdict(list)

    best_val_acc = 0

    for epoch in range(epochs):
        print(f'Epoch: {epoch + 1}/{epochs}')
        print('-' * 10)
        print('Training')
        train_acc, train_loss = train_epoch(model, train_loader, device, criterion, optimizer)
        print('\nValidating')
        val_acc, val_loss = val_epoch(model, val_loader, device, criterion)

        print(f'\nTrain Loss: {train_loss}\tTrain Acc: {train_acc}')
        print(f'Val Loss: {val_loss}\tVal Acc: {val_acc}')

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth.tar')

    return history

In [ ]:
EPOCHS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = CNN(num_classes=len(labels)).to(DEVICE)
history = train(model, EPOCHS, DEVICE, train_loader, val_loader, criterion, optimizer)

# Model Testing

In [ ]:
model.load_state_dict(torch.load('best_model.pth.tar'))

In [ ]:
test_df = pd.read_csv(root_dir + '/test.csv')
sample_sub_df = pd.read_csv(root_dir + '/sample_submission.csv')

In [ ]:
test_df.head()

In [ ]:
sample_sub_df.head()

In [ ]:
test_audio_dir = '../input/birdclef-2022/test_soundscapes'
for idx in tqdm(range(len(test_df))):
    audio_id = test_df.loc[idx, 'file_id']
    true_label = test_df.loc[idx, 'bird']
    end_time = test_df.loc[idx, 'end_time']
    
    path = os.path.join(test_audio_dir, audio_id, '.ogg')

    if os.path.isfile(path):
        sig, sr = torchaudio.load(file_pth)
        
        rows = sig.shape[1] // (32000 *5)
        sig = sig.reshape(rows, -1)

        row_id = end_time // 5

        sig = sig[row_id-1].reshape(1,-1)

        audio = MonoToStereo((sig, sr))
        audio = pad_signal(audio, 10000)
        audio = time_shift(audio, shift_limit=0.4)
        spec = mel_spec(audio)
        aug_spec = spectro_augment(spec)
        aug_spec = aug_spec.unsqueeze(0)
        output = model(aug_spec)

        _, pred = torch.max(output, dim=1)
        if labels[pred] == true_label:
            sample_sub_df.loc[idx, 'target'] = True
        else:
            sample_sub_df.loc[idx, 'target'] = False
        
    else:
        pred = True if random.randint(0,1) else False
        sample_sub_df.loc[idx, 'target'] = pred
        continue

# Make submission

In [ ]:
sample_sub_df.head()

In [ ]:
sample_sub_df.to_csv('submission.csv', index=False)

In [ ]:
print('Done!')